In [37]:
import pandas as pd
import numpy as np


df = pd.read_csv('train.csv')
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10836 entries, 0 to 10835
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   item_id   10836 non-null  object 
 1   year      10836 non-null  int64  
 2   month     10836 non-null  int64  
 3   seq       10836 non-null  float64
 4   type      10836 non-null  int64  
 5   hs4       10836 non-null  int64  
 6   weight    10836 non-null  float64
 7   quantity  10836 non-null  float64
 8   value     10836 non-null  float64
dtypes: float64(4), int64(4), object(1)
memory usage: 762.0+ KB


In [38]:
# ------------------------------------------------
# 1. 기본 컬럼 타입 정리
#    - year, month, hs4는 숫자
#    - value, weight, quantity는 float
# ------------------------------------------------
num_int_cols = ["year", "month", "hs4"]
for c in num_int_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

num_float_cols = ["value", "weight", "quantity"]
for c in num_float_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# ------------------------------------------------
# 2. date 컬럼 만들기 (year, month 기반)
# ------------------------------------------------
if "date" not in df.columns:
    df["date"] = pd.to_datetime(
        df["year"].astype(int).astype(str) + "-" +
        df["month"].astype(int).astype(str) + "-01"
    )
else:
    df["date"] = pd.to_datetime(df["date"])

print(df.head())
df.info()

    item_id  year  month  seq  type   hs4    weight  quantity     value  \
0  DEWLVASR  2022      1  1.0     1  3038   14858.0       0.0   32688.0   
1  ELQGMQWE  2022      1  1.0     1  2002   62195.0       0.0  110617.0   
2  AHMDUILJ  2022      1  1.0     1  2102   18426.0       0.0   72766.0   
3  XIPPENFQ  2022      1  1.0     1  2501   20426.0       0.0   11172.0   
4  FTSVTTSR  2022      1  1.0     1  2529  248000.0       0.0  143004.0   

        date  
0 2022-01-01  
1 2022-01-01  
2 2022-01-01  
3 2022-01-01  
4 2022-01-01  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10836 entries, 0 to 10835
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   item_id   10836 non-null  object        
 1   year      10836 non-null  int64         
 2   month     10836 non-null  int64         
 3   seq       10836 non-null  float64       
 4   type      10836 non-null  int64         
 5   hs4       10836 non-nul

In [39]:
# ------------------------------------------------
# 3. 거래 여부 플래그(row 단위)
#    - 이 row는 실제 거래 기록이 있다 = 1
#    - (value/weight/quantity가 다 NaN인 행은 제거하는 선택도 가능)
# ------------------------------------------------
df["has_tx"] = 1

# ------------------------------------------------
# 4. 월별 집계 (item_id + hs4 + date 기준)
#    - value/weight/quantity 합산
#    - has_tx는 row 개수 합산 후 0/1로 변환
#    - hs2, hs2_name_kr 같은 메타 정보는 "나중에 merge"에서 처리
# ------------------------------------------------
agg_dict = {
    "value":    lambda x: x.sum(min_count=1),   # 모두 NaN이면 NaN 유지
    "weight":   lambda x: x.sum(min_count=1),
    "quantity": lambda x: x.sum(min_count=1),
    "has_tx":   "sum",
}

group_cols = ["item_id", "hs4", "date"]

df_month = (
    df
    .groupby(group_cols, as_index=False)
    .agg(agg_dict)
)

# has_tx: 해당 (item_id, hs4, month)에 거래 row가 한 개 이상 있었으면 1
df_month["has_tx"] = (df_month["has_tx"] > 0).astype(int)

print(df_month.info())
df_month.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3776 entries, 0 to 3775
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   item_id   3776 non-null   object        
 1   hs4       3776 non-null   int64         
 2   date      3776 non-null   datetime64[ns]
 3   value     3776 non-null   float64       
 4   weight    3776 non-null   float64       
 5   quantity  3776 non-null   float64       
 6   has_tx    3776 non-null   int64         
dtypes: datetime64[ns](1), float64(3), int64(2), object(1)
memory usage: 206.6+ KB
None


,item_id,hs4,date,value,weight,quantity,has_tx
0,AANGBULD,4810,2022-01-01,14276.0,17625.0,0.0,1
1,AANGBULD,4810,2022-02-01,52347.0,67983.0,0.0,1
2,AANGBULD,4810,2022-03-01,53549.0,69544.0,0.0,1
3,AANGBULD,4810,2022-05-01,26997.0,34173.0,0.0,1
4,AANGBULD,4810,2022-06-01,84489.0,103666.0,0.0,1
5,AANGBULD,4810,2022-11-01,299.0,67.0,0.0,1
6,AANGBULD,4810,2022-12-01,574.0,66.0,0.0,1
7,AANGBULD,4810,2023-02-01,41353.0,50281.0,0.0,1
8,AANGBULD,4810,2023-05-01,26524.0,32345.0,0.0,1
9,AANGBULD,4810,2023-06-01,340.0,70.0,0.0,1


In [40]:
# ------------------------------------------------
# 5. 월단위 zero / missing 플래그 생성
#    - 집계된 value/weight/quantity 기준으로 다시 계산
# ------------------------------------------------
for col in ["value", "weight", "quantity"]:
    df_month[f"{col}_missing"] = df_month[col].isna().astype(int)
    # zero 플래그는 "NaN이 아닌데 값이 0인 경우"만 1
    df_month[f"{col}_zero"] = ((df_month[col] == 0) & df_month[col].notna()).astype(int)

print("월별 집계 + 플래그 예시:")
print(df_month.head())

# ------------------------------------------------
# 6. 전체 month 범위 & item_id 목록 만들기
#    - 풀 패널(빈 달 포함 전체 시계열) 생성을 위함
# ------------------------------------------------
min_month = df_month["date"].min()
max_month = df_month["date"].max()

full_months = pd.date_range(min_month, max_month, freq="MS")
all_items = df_month["item_id"].unique()

# ------------------------------------------------
# 7. item_id × date 전체 조합 (풀 패널 인덱스)
# ------------------------------------------------
panel = pd.MultiIndex.from_product(
    [all_items, full_months],
    names=["item_id", "date"]
).to_frame(index=False)

# ------------------------------------------------
# 8. 집계된 df_month와 merge해서 패널 완성
# ------------------------------------------------
df = (
    panel.merge(df_month, on=["item_id", "date"], how="left")
         .sort_values(["item_id", "date"])
         .reset_index(drop=True)
)

# merge 후: 거래 없는 month(has_tx가 NaN) → 0으로
df["has_tx"] = df["has_tx"].fillna(0).astype(int)

print(df.head(10))
df.info()

월별 집계 + 플래그 예시:
    item_id   hs4       date    value    weight  quantity  has_tx  \
0  AANGBULD  4810 2022-01-01  14276.0   17625.0       0.0       1   
1  AANGBULD  4810 2022-02-01  52347.0   67983.0       0.0       1   
2  AANGBULD  4810 2022-03-01  53549.0   69544.0       0.0       1   
3  AANGBULD  4810 2022-05-01  26997.0   34173.0       0.0       1   
4  AANGBULD  4810 2022-06-01  84489.0  103666.0       0.0       1   

   value_missing  value_zero  weight_missing  weight_zero  quantity_missing  \
0              0           0               0            0                 0   
1              0           0               0            0                 0   
2              0           0               0            0                 0   
3              0           0               0            0                 0   
4              0           0               0            0                 0   

   quantity_zero  
0              1  
1              1  
2              1  
3              1  

In [41]:

# ------------------------------------------------
# 9. "진짜 거래 없는 달"만 0으로 채우기
#    - has_tx == 0 인 경우만 value/weight/quantity = 0
#    - 거래 있는 달(has_tx == 1)의 NaN은 그대로 두어서
#      "측정 불가/미기재" 상태를 유지
# ------------------------------------------------
no_tx_mask = df["has_tx"] == 0

for col in ["value", "weight", "quantity"]:
    df.loc[no_tx_mask, col] = 0

# zero/missing 플래그도 거래 없는 달은 0으로 세팅
for col in [
    "value_zero", "weight_zero", "quantity_zero",
    "value_missing", "weight_missing", "quantity_missing",
]:
    df[col] = df[col].fillna(0).astype(int)

# ------------------------------------------------
# 10. year, month 컬럼 다시 생성
# ------------------------------------------------
df["year"] = df["date"].dt.year
df["month"] = df["date"].dt.month

print("최종 패널 예시:")
print(df.head())

print("중복 row 개수 (item_id, date 기준):",
      df.duplicated(["item_id", "date"]).sum())

최종 패널 예시:
    item_id       date     hs4    value   weight  quantity  has_tx  \
0  AANGBULD 2022-01-01  4810.0  14276.0  17625.0       0.0       1   
1  AANGBULD 2022-02-01  4810.0  52347.0  67983.0       0.0       1   
2  AANGBULD 2022-03-01  4810.0  53549.0  69544.0       0.0       1   
3  AANGBULD 2022-04-01     NaN      0.0      0.0       0.0       0   
4  AANGBULD 2022-05-01  4810.0  26997.0  34173.0       0.0       1   

   value_missing  value_zero  weight_missing  weight_zero  quantity_missing  \
0              0           0               0            0                 0   
1              0           0               0            0                 0   
2              0           0               0            0                 0   
3              0           0               0            0                 0   
4              0           0               0            0                 0   

   quantity_zero  year  month  
0              1  2022      1  
1              1  2022      2 

In [42]:
df.info()

df.to_csv("preprocessing.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4300 entries, 0 to 4299
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   item_id           4300 non-null   object        
 1   date              4300 non-null   datetime64[ns]
 2   hs4               3776 non-null   float64       
 3   value             4300 non-null   float64       
 4   weight            4300 non-null   float64       
 5   quantity          4300 non-null   float64       
 6   has_tx            4300 non-null   int64         
 7   value_missing     4300 non-null   int64         
 8   value_zero        4300 non-null   int64         
 9   weight_missing    4300 non-null   int64         
 10  weight_zero       4300 non-null   int64         
 11  quantity_missing  4300 non-null   int64         
 12  quantity_zero     4300 non-null   int64         
 13  year              4300 non-null   int32         
 14  month             4300 n